In [1]:
%pwd

'/home/user/work_space/RAG-Based-Conversational-Medical-Chatbot/research'

### Load PDF data 

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
# Extract data from PDF
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [4]:
extracted_data = load_pdf_files("../data")

In [5]:
print("Total pages: ", len(extracted_data))

Total pages:  637


### Filter operation to clean data

In [6]:
from typing import List
from langchain.schema import Document

In [7]:
def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """Given a list of document object, return a new list of document objects
    containing only 'source' in metadata and the original page content
    """
    minimal_docs : List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [8]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [9]:
print(len(minimal_docs))

637


### Split documents into smaller chuncks

In [10]:
def text_split_documents(documents: List[Document], chunk_size: int = 500, chunk_overlap: int = 20) -> List[Document]:
    """Split documents into smaller chunks using RecursiveCharacterTextSplitter."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    split_docs = text_splitter.split_documents(documents)
    return split_docs

In [11]:
text_chunk = text_split_documents(minimal_docs)
print("Total text chunks: ", len(text_chunk))

Total text chunks:  5859


### Get Embeddings through HuggingFace embedding models

In [12]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

def download_embeddings_model(model_name: str = "sentence-transformers/all-MiniLM-L6-v2") -> HuggingFaceBgeEmbeddings:
    """Download and return the HuggingFace BGE embeddings model."""
    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name      
    )
    return embeddings   

In [13]:
embedding = download_embeddings_model()
print("Embeddings model downloaded.")

/tmp/ipykernel_92089/705367593.py:5: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


Embeddings model downloaded.


In [14]:
vector = embedding.embed_query("Hello, world!")   
print("Sample embedding vector: ", vector[:5])  # Print first 5 dimensions of the embedding vector

Sample embedding vector:  [0.003967964090406895, 0.1618259847164154, 0.03274252638220787, -0.010052550584077835, -0.02936175838112831]


In [15]:
print("vector length: ", len(vector))

vector length:  384


### Load Pinecone from dotenv

In [16]:
from dotenv import load_dotenv
import os
load_dotenv()  

True

In [17]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY   

In [18]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

### Create Index

In [20]:
from pinecone import ServerlessSpec

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [21]:
index = pc.Index(index_name)

In [22]:
# 5. Create Pinecone VectorStore from documents
from langchain_pinecone import PineconeVectorStore

try:
    dosearch = PineconeVectorStore.from_documents(
        documents=text_chunk,
        embedding=embedding,
        index_name=index_name
    )
    print("✅ Pinecone VectorStore created successfully.")
except Exception as e:
    print(f"❌ Error creating Pinecone VectorStore: {e}")

✅ Pinecone VectorStore created successfully.


## Load Existing Index

In [23]:


from langchain_pinecone import PineconeVectorStore  
existing_vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)
print("✅ Existing Pinecone VectorStore loaded successfully.")

✅ Existing Pinecone VectorStore loaded successfully.


### Add more data to the existing index

In [24]:
dswith = Document( 
    page_content="There is a trinity in hinduism about brahma vishnu and shiva",
    metadata={"source": "wikipedia.com"}
)

In [25]:
index_id = dosearch.add_documents(documents=[dswith])

In [26]:
print("Document added with ID: ", index_id)

Document added with ID:  ['95ac8e67-3094-4550-9d34-89ab9ce20309']


In [27]:
# Create retriever
retriever = existing_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [28]:
retrieved_docs = retriever.invoke("Tell me about hinduism") 
print(retrieved_docs[0])

page_content='There is a trinity in hinduism about brahma vishnu and shiva' metadata={'source': 'wikipedia.com'}


### Import LLM 

In [29]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o", temperature=0) 

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [31]:
system_prompt = {
    "You are a helpful medical assistant. "
    "Use the following context to answer the question at the end. "
    "If you don't know the answer, just say that you don't know, don't try to make up an answer"
    "dont know. Use three sentences maximum to answer the question precisely."
    "\n\n\n"
    "{context}"
}

In [32]:
prompt = ChatPromptTemplate(
    messages=system_prompt,
    input_variables=["context", "question"]     
)

In [33]:
question_answer_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [37]:
# checking the RAG chain
response = rag_chain.invoke({"input" : "Tell me about hinduism"})
print(response["Answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}